In [45]:
from atproto_client.models.app.bsky.actor.defs import ProfileView
from atproto_client.models.app.bsky.feed.defs import ThreadViewPost
from atproto_client.models.app.bsky.feed.post import GetRecordResponse
from atproto_client.models.app.bsky.feed.get_likes import Like
from atproto_client.models.app.bsky.feed.get_post_thread import Response as PostThreadResponse

import pandas as pd

from lib.helper import client
from services.sync.search.helper import send_request_with_pagination

In [2]:
civic_posts = pd.read_csv('sample_bluesky_civic_posts.csv')
noncivic_posts = pd.read_csv('sample_bluesky_noncivic_posts.csv')

Let's grab the most up-to-date engagement data for each post. We have the link for each post so we can easily get the post ID. Let's grab the post record object given the link for the post.

In [3]:
def get_author_handle_and_post_id_from_link(link: str) -> dict[str, str]:
    """Given a link, get the author and post ID.
    
    Example:
    >get_author_and_post_id_from_link("https://bsky.app/profile/scottsantens.com/post/3knqkh2es7k2i")
    {'author': 'scottsantens.com', 'post_id': '3knqkh2es7k2i}
    """
    # Split the link by the forward slash
    split_link = link.split("/")
    # Get the author and post ID
    author = split_link[4]
    post_id = split_link[6]
    return {"author": author, "post_id": post_id}

In [4]:
def get_author_did_from_handle(author_handle: str) -> str:
    """Given an author handle, get the DID.
    
    Example:
    >get_author_did_from_handle("scottsantens.com")
    "did:example:123"
    """
    # Get the profile
    profile = client.get_profile(author_handle)
    # Get the DID
    return profile["did"]


In [5]:
def get_post_record_from_post_link(link: str) -> GetRecordResponse:
    """Given a post link, get the post record.
    
    Example:
    >post = get_post_record_from_post_link("https://bsky.app/profile/gbbranstetter.bsky.social/post/3knssi4ouko24")
    GetRecordResponse(
        uri='at://did:plc:mlmouohgzbjofidukcp4pxf2/app.bsky.feed.post/3knssi4ouko24',
        value=Record(created_at='2024-03-16T12:17:36.784Z', text='A running theme in Woods\' telling is how even those who opposed Hitler blamed his rise on the "depravity" and excess of Weimar Berlin "as if the mere presence of ersatz women in the club was enough to foment and justify a right-wing putsch"--a phrase I find myself repeating often', embed=Main(record=Main(cid='bafyreiesskdi2vfkrvj2kaajglsklqd7b3wywn2goyxhr7ctl3kvaikbsa', uri='at://did:plc:mlmouohgzbjofidukcp4pxf2/app.bsky.feed.post/3knss3jmoeu2f', py_type='com.atproto.repo.strongRef'), py_type='app.bsky.embed.record'), entities=None, facets=None, labels=None, langs=['en'], reply=None, tags=None, py_type='app.bsky.feed.post'),
        cid='bafyreidujr2qzblrtxyh6e5shqtjdool2c7vim5jsqybdhi4pkpuphuj5q'
    )
    """
    author_and_post_id: dict = get_author_handle_and_post_id_from_link(link)
    author_did: str = get_author_did_from_handle(author_and_post_id["author"])
    post_rkey = author_and_post_id["post_id"]
    profile_identify = author_did
    print(f"Getting post record for {post_rkey} by {profile_identify}")
    response = client.get_post(
        post_rkey=post_rkey, profile_identify=profile_identify
    )
    return response

In [6]:
post_record = get_post_record_from_post_link("https://bsky.app/profile/gbbranstetter.bsky.social/post/3knssi4ouko24")

HTTP Request: GET https://bsky.social/xrpc/app.bsky.actor.getProfile?actor=gbbranstetter.bsky.social "HTTP/1.1 200 OK"


Getting post record for 3knssi4ouko24 by did:plc:mlmouohgzbjofidukcp4pxf2


HTTP Request: GET https://bsky.social/xrpc/com.atproto.repo.getRecord?collection=app.bsky.feed.post&repo=did%3Aplc%3Amlmouohgzbjofidukcp4pxf2&rkey=3knssi4ouko24 "HTTP/1.1 200 OK"


In [7]:
def get_repost_profiles(post_uri: str) -> list[ProfileView]:
    """Get the profiles of all the users who reposted a post."""
    reposts: list[ProfileView] = send_request_with_pagination(
        func=client.get_reposted_by,
        kwargs={"uri": post_uri},
        response_key="reposted_by",
        limit=None
    )

    return reposts

In [16]:
def get_liked_by_profiles(post_uri: str) -> list[Like]:
    """Get the profiles of all the users who liked a post."""
    likes: list[Like] = send_request_with_pagination(
        func=client.get_likes,
        kwargs={"uri": post_uri},
        response_key="likes",
        limit=None
    )

    return likes

In [47]:
def get_post_thread_replies(post_uri: str) -> list[ThreadViewPost]:
    """Get the thread of replies to a post."""
    response: PostThreadResponse = client.get_post_thread(post_uri["uri"])
    thread: ThreadViewPost = response.thread
    replies: list[ThreadViewPost] = thread.replies
    return replies

In [ ]:
def calculate_post_engagement(post_response: GetRecordResponse) -> dict:
    """Calculates the number of likes, retweets, and comments for a post."""
    # grab # of likes
    likes: list[Like] = get_liked_by_profiles(post_response["uri"])
    num_likes: int = len(likes)

    # grab # of retweets/reposts
    reposts: list[ProfileView] = get_repost_profiles(post_response["uri"])
    num_reposts = len(reposts)

    # grab # of replies
    replies: list[ThreadViewPost] = get_post_thread_replies(post_response["uri"])
    num_replies = len(replies)

    return {
        "uri": post_response["uri"],
        "num_likes": num_likes,
        "num_reposts": num_reposts,
        "num_replies": num_replies
    }


Need to grab engagement:
- Number of likes
- Number of retweets
- Number of comments

Need to also later on sort by timestamp

In [56]:
type(response.__dict__["value"])

atproto_client.models.app.bsky.feed.post.Record

In [94]:
likes_response = client.get_likes("at://did:plc:mlmouohgzbjofidukcp4pxf2/app.bsky.feed.post/3knssi4ouko24")

HTTP Request: GET https://bsky.social/xrpc/app.bsky.feed.getLikes?uri=at%3A%2F%2Fdid%3Aplc%3Amlmouohgzbjofidukcp4pxf2%2Fapp.bsky.feed.post%2F3knssi4ouko24 "HTTP/1.1 200 OK"


In [96]:
len(likes_response.likes)

50

In [47]:
response.__dict__

{'uri': 'at://did:plc:mlmouohgzbjofidukcp4pxf2/app.bsky.feed.post/3knssi4ouko24',
 'value': Record(created_at='2024-03-16T12:17:36.784Z', text='A running theme in Woods\' telling is how even those who opposed Hitler blamed his rise on the "depravity" and excess of Weimar Berlin "as if the mere presence of ersatz women in the club was enough to foment and justify a right-wing putsch"--a phrase I find myself repeating often', embed=Main(record=Main(cid='bafyreiesskdi2vfkrvj2kaajglsklqd7b3wywn2goyxhr7ctl3kvaikbsa', uri='at://did:plc:mlmouohgzbjofidukcp4pxf2/app.bsky.feed.post/3knss3jmoeu2f', py_type='com.atproto.repo.strongRef'), py_type='app.bsky.embed.record'), entities=None, facets=None, labels=None, langs=['en'], reply=None, tags=None, py_type='app.bsky.feed.post'),
 'cid': 'bafyreidujr2qzblrtxyh6e5shqtjdool2c7vim5jsqybdhi4pkpuphuj5q'}

In [39]:
post = get_post_record_from_post_link("https://bsky.app/profile/gbbranstetter.bsky.social/post/3knssi4ouko24")

HTTP Request: GET https://bsky.social/xrpc/app.bsky.actor.getProfile?actor=gbbranstetter.bsky.social "HTTP/1.1 200 OK"


Getting post record for 3knssi4ouko24 by did:plc:mlmouohgzbjofidukcp4pxf2


HTTP Request: GET https://bsky.social/xrpc/com.atproto.repo.getRecord?collection=app.bsky.feed.post&repo=did%3Aplc%3Amlmouohgzbjofidukcp4pxf2&rkey=3knssi4ouko24 "HTTP/1.1 200 OK"


In [43]:
record.__dict__

{'created_at': '2024-02-08T10:10:09.453Z',
 'text': "Oh no! How dare you, pronoucing your opinion about politics which might be influencing your very life. In a public space? Willy-nilly? Not on that guy's watch.",
 'embed': Main(images=[Image(alt='', image=BlobRef(mime_type='image/jpeg', size=49277, ref=IpldLink(link='bafkreihxwhsyvjvfi5ttp73ctrzqxkpnhk4kvgiqaxxoqbuerdm45p4dn4'), py_type='blob'), aspect_ratio=AspectRatio(height=197, width=256, py_type='app.bsky.embed.images#aspectRatio'), py_type='app.bsky.embed.images#image')], py_type='app.bsky.embed.images'),
 'entities': None,
 'facets': None,
 'labels': None,
 'langs': ['cs'],
 'reply': ReplyRef(parent=Main(cid='bafyreifdjsjf27qcmoa4ajdrzqvk62pmokm5pyyvrnw2cd3lw5hcz5kzue', uri='at://did:plc:atwgx3kry3qwsd67eve5wkwd/app.bsky.feed.post/3kkvjexa6nl2e', py_type='com.atproto.repo.strongRef'), root=Main(cid='bafyreic3xabs2jpcaer7ppk7zzuv55batlenxynjgtekd6mcjwbinmottu', uri='at://did:plc:atwgx3kry3qwsd67eve5wkwd/app.bsky.feed.post/3kkrk

In [4]:
civic_posts.head()

,Link,Text,Retweets,Likes,Comments,Feed,Timestamp,Goal: collect attention-grabbing political tweets
0,https://bsky.app/profile/gbbranstetter.bsky.so...,A running theme in Woods' telling is how even ...,46,177.0,NaN,https://bsky.app/profile/did:plc:z72i7hdynmk6r...,3/16/2024,NaN
1,https://bsky.app/profile/juliusgoat.bsky.socia...,And now conservatives today are making the sam...,52,151.0,NaN,https://bsky.app/profile/did:plc:z72i7hdynmk6r...,3/16/2024,NaN
2,https://bsky.app/profile/jbouie.bsky.social/po...,this from @radleybalko.bsky.social is very good,79,225.0,NaN,https://bsky.app/profile/did:plc:z72i7hdynmk6r...,3/16/2024,NaN
3,https://bsky.app/profile/gbbranstetter.bsky.so...,They say this about Putin and Orban and their ...,15,58.0,NaN,https://bsky.app/profile/did:plc:z72i7hdynmk6r...,3/16/2024,NaN
4,https://bsky.app/profile/jameeljaffer.bsky.soc...,This framing is really something.,20,139.0,NaN,https://bsky.app/profile/did:plc:z72i7hdynmk6r...,3/16/2024,NaN


In [3]:
civic_posts_dicts = civic_posts.to_dict(orient='records')
noncivic_posts_dicts = noncivic_posts.to_dict(orient='records')